In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32482
2,Huelva,Confirmados PDIA 14 días,1487
3,Huelva,Tasa PDIA 14 días,"289,76752343278054"
4,Huelva,Confirmados PDIA 7 días,652
5,Huelva,Total Confirmados,32663
6,Huelva,Curados,30170
7,Huelva,Fallecidos,386


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32482.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8829.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 470 personas en los últimos 7 días 

Un positivo PCR cada 227 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32482.0,652.0,1487.0,513170.0,127.053413,289.767523,276.0
Huelva-Costa,18731.0,424.0,874.0,289548.0,146.435133,301.849780,179.0
Huelva (capital),8829.0,306.0,635.0,143837.0,212.740811,441.471944,130.0
Condado-Campiña,10532.0,177.0,464.0,156231.0,113.293777,296.996115,77.0
Sierra de Huelva-Andévalo Central,2868.0,50.0,144.0,67391.0,74.193883,213.678384,19.0
San Juan del Puerto,550.0,27.0,56.0,9411.0,286.898310,595.048348,16.0
Moguer,1348.0,38.0,78.0,21867.0,173.777839,356.701880,12.0
Bollullos Par del Condado,1015.0,16.0,43.0,14387.0,111.211510,298.880934,10.0
Palma del Condado (La),1174.0,11.0,31.0,10801.0,101.842422,287.010462,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,13.0,5.0,5.0,299.0,1672.240803,1672.240803,1.0
Puebla de Guzmán,132.0,12.0,19.0,3092.0,388.098318,614.489004,7.0
San Juan del Puerto,550.0,27.0,56.0,9411.0,286.898310,595.048348,16.0
Villarrasa,180.0,3.0,13.0,2211.0,135.685210,587.969245,0.0
Valverde del Camino,627.0,27.0,74.0,12750.0,211.764706,580.392157,7.0
Palos de la Frontera,696.0,15.0,54.0,11742.0,127.746551,459.887583,6.0
Huelva (capital),8829.0,306.0,635.0,143837.0,212.740811,441.471944,130.0
Niebla,247.0,8.0,16.0,4116.0,194.363460,388.726919,6.0
Moguer,1348.0,38.0,78.0,21867.0,173.777839,356.701880,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,128.0,1.0,10.0,3812.0,26.232949,262.329486,0.0,0.100000
Rociana del Condado,559.0,2.0,12.0,7939.0,25.192090,151.152538,1.0,0.166667
Manzanilla,103.0,1.0,5.0,2118.0,47.214353,236.071766,1.0,0.200000
Villarrasa,180.0,3.0,13.0,2211.0,135.685210,587.969245,0.0,0.230769
Almonte,1864.0,18.0,71.0,24507.0,73.448402,289.713143,5.0,0.253521
Palos de la Frontera,696.0,15.0,54.0,11742.0,127.746551,459.887583,6.0,0.277778
Trigueros,324.0,7.0,24.0,7862.0,89.035869,305.265836,2.0,0.291667
Nerva,179.0,1.0,3.0,5169.0,19.346102,58.038305,0.0,0.333333
Cartaya,1363.0,12.0,35.0,20083.0,59.752029,174.276751,6.0,0.342857
